In [ ]:
!pip install numpy pandas scikit-surprise sklearn seaborn matplotlib spacy nltk navec slovnet natasha

     |████████████████████████████████| 34.4MB 109kB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 8.2MB 19.5MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26102 sha256=966842acb52b8dbc4ae3d66fd3f7e7d9f5db37be0860433922f8fc92f2b7d6a2
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


# **Токенизация. NLTK**

In [ ]:
import nltk
nltk.download('punkt')
text1 = 'Предобработка данных в XML файле.'
text2 = 'Меня зовут Бонд. Джеймс Бонд'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from nltk import tokenize
dir(tokenize)[:18]

['BlanklineTokenizer',
 'LineTokenizer',
 'MWETokenizer',
 'PunktSentenceTokenizer',
 'RegexpTokenizer',
 'ReppTokenizer',
 'SExprTokenizer',
 'SpaceTokenizer',
 'StanfordSegmenter',
 'TabTokenizer',
 'TextTilingTokenizer',
 'ToktokTokenizer',
 'TreebankWordTokenizer',
 'TweetTokenizer',
 'WhitespaceTokenizer',
 'WordPunctTokenizer',
 '__builtins__',
 '__cached__']

In [ ]:
nltk_tk_1 = nltk.WordPunctTokenizer()
nltk_word = nltk_tk_1.tokenize(text1)
print(nltk_word)

['Предобработка', 'данных', 'в', 'XML', 'файле', '.']


In [ ]:
# Токенизация по предложениям
nltk_tk_sents = nltk.tokenize.sent_tokenize(text1)
print(len(nltk_tk_sents))
nltk_tk_sents

1


['Предобработка данных в XML файле.']

# **Частеречная разметка. Natasha**

In [ ]:
from navec import Navec
from slovnet import Morph

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
navec = Navec.load('/content/gdrive/My Drive/MMO/navec_news_v1_1B_250K_300d_100q.tar')
n_morph = Morph.load('/content/gdrive/My Drive/MMO/slovnet_morph_news_v1.tar', batch_size=4)

In [ ]:
morph_res = n_morph.navec(navec)

In [ ]:
def print_pos(markup):
    for token in markup.tokens:
        print('{} - {}'.format(token.text, token.tag))

In [ ]:
n_text1_markup = list(_ for _ in n_morph.map(nltk_tk_sents))
[print_pos(x) for x in n_text1_markup]

П - PROPN|Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
р - NOUN
е - X|Foreign=Yes
д - NOUN
о - X|Foreign=Yes
б - NOUN|Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing
р - X|Foreign=Yes
а - CCONJ
б - PROPN
о - NOUN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
т - PRON|Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing
к - ADP
а - X|Foreign=Yes
  - PUNCT
д - NOUN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
а - CCONJ
н - X|Foreign=Yes
н - X|Foreign=Yes
ы - X|Foreign=Yes
х - X|Foreign=Yes
  - PUNCT
в - X|Foreign=Yes
  - PUNCT
X - X|Foreign=Yes
M - PROPN|Foreign=Yes
L - X|Foreign=Yes
  - PUNCT
ф - X|Foreign=Yes
а - CCONJ
й - ADJ|Case=Nom|Degree=Pos|Gender=Masc|Number=Sing
л - X|Foreign=Yes
е - NOUN|Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
. - PUNCT


[None]

# **Лемматизация. Natasha**

In [ ]:
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab

In [ ]:
def n_lemmatize(text):
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    segmenter = Segmenter()
    morph_vocab = MorphVocab()
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return doc

In [ ]:
n_doc1 = n_lemmatize(text1)
{_.text: _.lemma for _ in n_doc1.tokens}

{'.': '.',
 'XML': 'xml',
 'Предобработка': 'предобработка',
 'в': 'в',
 'данных': 'данные',
 'файле': 'файл'}

In [ ]:
n_doc2 = n_lemmatize(text2)
{_.text: _.lemma for _ in n_doc2.tokens}

{'.': '.', 'Бонд': 'бонд', 'Джеймс': 'джеймс', 'Меня': 'я', 'зовут': 'звать'}

# **Выделение (распознавание) именованных сущностей. Natasha**

In [ ]:
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup

In [ ]:
ner = NER.load('/content/gdrive/My Drive/MMO/slovnet_ner_news_v1.tar')

In [ ]:
ner_res = ner.navec(navec)

In [ ]:
markup_ner2 = ner(text2)

In [ ]:
markup_ner2

SpanMarkup(
    text='Меня зовут Бонд. Джеймс Бонд',
    spans=[Span(
         start=11,
         stop=15,
         type='PER'
     ), Span(
         start=17,
         stop=28,
         type='PER'
     )]
)

In [ ]:
show_markup(markup_ner2.text, markup_ner2.spans)

Меня зовут Бонд. Джеймс Бонд
           PER─  PER────────


# **Разбор предложения. Natasha**

In [ ]:
from natasha import NewsSyntaxParser

In [ ]:
emb = NewsEmbedding()
syntax_parser = NewsSyntaxParser(emb)

In [ ]:
n_doc1.parse_syntax(syntax_parser)
n_doc1.sents[0].syntax.print()

┌► Предобработка amod
└─ данных        
┌► в             case
└─ XML           
   файле         
   .             


In [ ]:
n_doc2.parse_syntax(syntax_parser)
n_doc2.sents[0].syntax.print()

    ┌► Меня  obj
┌─┌─└─ зовут 
│ └──► Бонд  xcomp
└────► .     punct
